In [12]:
import os

In [9]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
save_folder = os.path.join(project_root, 'local_only')
os.makedirs(save_folder, exist_ok=True)

In [10]:
dataset_url = 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_test_fullwiki_v1.json'
file_name = dataset_url.split('/')[-1]
output_path = os.path.join(save_folder, file_name)

In [11]:
%%time

import subprocess
subprocess.run(["curl", "-L", "-o", output_path, dataset_url], check=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 82 44.0M   82 36.1M    0     0  5909k      0  0:00:07  0:00:06  0:00:01 7597k

CPU times: user 7.36 ms, sys: 0 ns, total: 7.36 ms
Wall time: 7 s


100 44.0M  100 44.0M    0     0  6479k      0  0:00:06  0:00:06 --:--:-- 9630k


CompletedProcess(args=['curl', '-L', '-o', '/mnt/f/chatbot_ui_v2/local_only/hotpot_test_fullwiki_v1.json', 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_test_fullwiki_v1.json'], returncode=0)

In [22]:
%%time

dataset_url = 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json'
file_name = dataset_url.split('/')[-1]
output_path = os.path.join(save_folder, file_name)

import subprocess
subprocess.run(["curl", "-L", "-o", output_path, dataset_url], check=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 88 45.2M   88 40.2M    0     0  6623k      0  0:00:06  0:00:06 --:--:-- 8512k

CPU times: user 6.68 ms, sys: 990 μs, total: 7.67 ms
Wall time: 6.96 s


100 45.2M  100 45.2M    0     0  6689k      0  0:00:06  0:00:06 --:--:-- 9204k


CompletedProcess(args=['curl', '-L', '-o', '/mnt/f/chatbot_ui_v2/local_only/hotpot_dev_fullwiki_v1.json', 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json'], returncode=0)

In [17]:
import json

In [24]:
# Open and load JSON content
file =  '/mnt/f/chatbot_ui_v2/local_only/hotpot_test_fullwiki_v1.json'
with open(file, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [25]:
type(data), len(data)

(list, 7405)

In [27]:
# Open and load JSON content
file =  '/mnt/f/chatbot_ui_v2/local_only/hotpot_dev_fullwiki_v1.json'
with open(file, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [28]:
type(data), len(data)

(list, 7405)

In [55]:
def process_record(record):
    data = dict(record)
    context = data.pop('context')
    condensed_context = condense_context(context)
    _id = data.pop('_id')
    supporting_facts = data.pop('supporting_facts')
    supporting_facts = list({i[0] for i in supporting_facts})
    metadata = dict(answer = data.pop('answer'), type = data.pop('type'), level=data.pop('level'), 
                    question = data.pop('question'), supporting_facts=supporting_facts)

    documents = list(condensed_context.values())
    ids = [f'{_id}_{i}' for i in range(len(documents))]
    metadatas = [dict(**metadata, title=title)   for title in condensed_context.keys()]
    
    return dict(documents=documents, metadatas=metadatas, ids=ids)

In [58]:
process_record(data[1])

{'documents': ['A Kiss for Corliss is a 1949 American comedy film directed by Richard Wallace and written by Howard Dimsdale. It stars Shirley Temple in her final starring role as well as her final film appearance. It is a sequel to the 1945 film "Kiss and Tell". "A Kiss for Corliss" was retitled "Almost a Bride" before release and this title appears in the title sequence. The film was released on November 25, 1949, by United Artists.',
  'The post of Lord High Treasurer or Lord Treasurer was an English government position and has been a British government position since the Acts of Union of 1707. A holder of the post would be the third-highest-ranked Great Officer of State, below the Lord High Steward and the Lord High Chancellor.',
  'Meet Corliss Archer is an American television sitcom that aired on CBS (July 13, 1951 - August 10, 1951) and in syndication via the Ziv Company from April to December 1954. The program was an adaptation of the radio series of the same name, which was ba

In [43]:
def condense_context(context):
    return {i[0]: ''.join(i[1]) for i in context}

In [59]:
import chromadb

In [60]:
client = chromadb.HttpClient(
    host="localhost",
    port=int(8010))

In [85]:
#dir(client)

In [83]:
for collection in client.list_collections():
    print(collection, collection.count())

Collection(name=test_collection) 50
Collection(name=hot_pot_qa_v2) 73642
Collection(name=hot_pot_qa) 4627


In [68]:
collection = client.get_collection('test_collection')

In [70]:
collection.id

UUID('935215a8-24d7-466b-ac31-400c97df58cb')

In [79]:
#collection.get()

In [76]:
abc = collection.get(include=["embeddings"])

In [87]:
# Get 1 record (equivalent to SELECT * LIMIT 1)
results = collection.get(limit=1, include=["documents", "metadatas"])
print(results)

{'ids': ['5a8b57f25542995d1e6f1371_0'], 'embeddings': None, 'metadatas': [{'answer': 'yes', 'type': 'comparison', 'title': 'Adam Collis', 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'supporting_facts': 'Ed Wood||Scott Derrickson', 'level': 'hard'}], 'documents': ['Adam Collis is an American filmmaker and actor. He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010. He also studied cinema at the University of Southern California from 1991 to 1997. Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995). In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".'], 'data': None, 'uris': None, 'included': ['documents', 'metadatas']}


In [88]:
results = collection.get(limit=10,
    where={"level": "easy"},
    include=["documents", "metadatas"]
)

In [89]:
results

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None,
 'included': ['documents', 'metadatas']}